# Dacon Competition Project : Predict Market Sales

## Data field
* store_id : 상점의 고유 아이디
* card_id : 사용한 카드의 고유 아이디
* card_company : 비식별화된 카드 회사
* transacted_date: 거래 날짜
* transacted_time : 거래 시간 (시:분)
* installment_term : 할부 개월 수( 포인트 사용 시 (60개월 + 실제할부개월)을 할부개월수에 기재한다. )
* region : 상점의 지역
* type_of_business:상점의 업종
* amount : 거래액(단위는 원이 아닙니다)

## Index
### Step1. Data Load & Resampling
* 시계열 분석을 위한 date index로 변환
* 시간 단위로 나뉘어져 있는 데이터를 일 단위로 resampling
* Modeling을 위해 월 단위로 resampling


In [1]:
# jupyter notebook cell 너비 조절
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 기본
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.externals import joblib 
from sklearn.metrics import make_scorer

# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Deep Neural Network
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time

# 설정
%matplotlib inline
pd.options.display.max_columns = 400
pd.options.display.float_format = '{:.5f}'.format


ModuleNotFoundError: No module named 'fbprophet'

## Evaluate Metric
* MAE (Mean Absolute Error) : 절댓값 오차의 평균

$MAE=\frac{1}{𝑛}∑| 𝑦𝑖−𝑦̂𝑖|$
 
* validation을 위한 함수 생성 및 make scorer

In [3]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

## Issue
1) 예측해야 하는 범위는 3개월인데 데이터는 시간 단위로 나뉘어져 있음
* month 주기로 resampling 후 forecast 범위를 3개월로 지정하여 해결

2) 1967개의 store_id가 각각의 trend와 seasonality를 가지고 있음
* 같은 알고리즘에서 각 store_id별로 parameter를 조절한다.

3) 예측 날짜는 2019-03~2019-05로 동일하나, 제공 데이터의 마지막 날짜는 차이가 있다.
* 마지막 날짜부터 3개월만 예측하여 제출한다.(ex: store_id 111의 마지막 날짜는 2018-09월로 뒤 3개월인 2018-10~2018-12만 예측하여 제출)
* 예측 기간이 길어질수록 오차가 크게 발생하여 바로 뒤 3개월만 예측하는 것이 정확도가 높았음

# Step1. Data Load

In [4]:
df_train = pd.read_csv('./data/funda_train.csv')
df_sub = pd.read_csv('./data/submission.csv')
df_train['transacted_date'] = pd.to_datetime(df_train['transacted_date'])

print(df_train.shape)

(6556613, 9)


In [39]:
df_train['transacted_date'].dtype

dtype('<M8[ns]')

In [40]:
df_train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [19]:
# 각 컬럼의 유니크한 값이 몇개인지, 결측치 개수, 데이터타입, 유니크한 값 5개
### 아래 for문 풀어서 써보기 !
frame_info=[ ( col, len(df_train[col].unique()), df_train[col].isnull().sum(), df_train[col].dtype, df_train[col].unique()[:5] ) for col in df_train.columns]
df_info=pd.DataFrame(frame_info, columns=['name', 'num_of_unique', 'num_of_nan', 'type', 'front5_values'])
df_info

,name,num_of_unique,num_of_nan,type,front5_values
0,store_id,1967,0,int64,"[0, 1, 2, 4, 5]"
1,card_id,3950001,0,int64,"[0, 1, 2, 3, 4]"
2,card_company,8,0,object,"[b, h, c, a, f]"
3,transacted_date,1003,0,datetime64[ns],"[2016-06-01T00:00:00.000000000, 2016-06-02T00:..."
4,transacted_time,1440,0,object,"[13:13, 18:12, 18:52, 20:22, 11:06]"
5,installment_term,34,0,int64,"[0, 2, 3, 60, 4]"
6,region,181,2042766,object,"[nan, 서울 종로구, 충북 충주시, 부산 동래구, 경기 평택시]"
7,type_of_business,146,3952609,object,"[기타 미용업, nan, 의복 액세서리 및 모조 장신구 도매업, 한식 음식점업, 배..."
8,amount,30551,0,float64,"[1857.142857142857, 857.1428571428571, 2000.0,..."


### 시계열 분석을 위해 date 정보를 index로 변환

In [41]:
df_train = df_train.set_index('transacted_date')
df_train.head(3)

,store_id,card_id,card_company,transacted_time,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,
2016-06-01,0,0,b,13:13,0,NaN,기타 미용업,1857.142857
2016-06-01,0,1,h,18:12,0,NaN,기타 미용업,857.142857
2016-06-01,0,2,c,18:52,0,NaN,기타 미용업,2000.000000


### 시간 단위로 나뉘어져 있는 데이터를 일단위로 resampling
* 'store_id', 'region', 'type_of_business' 기존과 동일
* day_of_week : 각 요일을 나타내는 숫자, 월요일은 0 일요일은 6
* business_day : working day 여부, 1이면 working day 0이면 주말
* num_of_pay : 일 결제 건수, 'card_id'의 count로 생성
* num_of_revisit : 단골 방문 횟수, 'card_id'의 value 중 count가 2보다 큰(3 이상) value의 결제 건수
* installment_term : 일 총 할부 개월 수, 기존 installment_term의 합
* amount : 일 매출 액, 기존 amount의 합

> 시간별 데이터를 일별로 카운트하고 싶을 때
* resample(rule='d').count( ) <br>

> 시간별 데이터를 일별로 합계를 구하고 싶을 때
* resample(rule='d').sum( )

In [49]:
df_train[df_train.store_id==0]

,store_id,card_id,card_company,transacted_time,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,
2016-06-01,0,0,b,13:13,0,NaN,기타 미용업,1857.142857
2016-06-01,0,1,h,18:12,0,NaN,기타 미용업,857.142857
2016-06-01,0,2,c,18:52,0,NaN,기타 미용업,2000.000000
2016-06-01,0,3,a,20:22,0,NaN,기타 미용업,7857.142857
2016-06-02,0,4,c,11:06,0,NaN,기타 미용업,2000.000000
...,...,...,...,...,...,...,...,...
2019-02-28,0,1476,a,12:17,0,NaN,기타 미용업,2857.142857
2019-02-28,0,1719,a,16:20,0,NaN,기타 미용업,6428.571429
2019-02-28,0,1791,b,16:56,0,NaN,기타 미용업,7142.857143


#### resample() 함수
    * resample('d') 는 '년-월-일 시간:분:초' 의 시계열 index를 1일 단위의 동일 간격별로 데이터를 뽑으라는 뜻
#### insert() 함수
    * insert(a, b)는 리스트의 a번째 위치에 b를 삽입하는 함수이다.
#### dayofweek
    * date정보를 index로 변환했기 때문에 
    * df_train.index.dayofweek를 하면 각 요일을 나타내는 숫자 출력됨

In [139]:
df_train.index.dayofweek

Int64Index([2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
            ...
            3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
           dtype='int64', name='transacted_date', length=6556613)

In [69]:
df_train[df_train.store_id==0]['card_id'].resample(rule='d').count().rename('num_of_pay')

transacted_date
2016-06-01    4
2016-06-02    7
2016-06-03    3
2016-06-04    7
2016-06-05    3
             ..
2019-02-24    6
2019-02-25    6
2019-02-26    0
2019-02-27    5
2019-02-28    6
Freq: D, Name: num_of_pay, Length: 1003, dtype: int64

In [90]:
df_train[df_train.store_id==0].card_id.value_counts().reset_index().query("card_id >2")["index"].values[:10]

array([ 32, 855,  44, 669, 898, 138, 480, 560, 346, 595])

In [95]:
df_train[df_train.card_id.isin(df_train[df_train.store_id==0].card_id.value_counts().reset_index().query("card_id >2")["index"].values)].card_id

transacted_date
2016-06-01       0
2016-06-01       1
2016-06-01       2
2016-06-01       3
2016-06-02       4
              ... 
2018-04-28    1248
2017-10-04    1252
2017-12-22     117
2018-03-12     488
2017-02-11     707
Name: card_id, Length: 2877, dtype: int64

In [98]:
df_train[df_train.store_id==0][['installment_term','amount']].resample(rule='d').sum()

,installment_term,amount
transacted_date,,
2016-06-01,0,12571.428571
2016-06-02,0,40571.428571
2016-06-03,0,18142.857143
2016-06-04,0,31714.285714
2016-06-05,0,10428.571429
...,...,...
2019-02-24,0,38571.428571
2019-02-25,3,39714.285714
2019-02-26,0,0.000000


In [122]:
df_train[df_train.store_id==0].type_of_business

transacted_date
2016-06-01    기타 미용업
2016-06-01    기타 미용업
2016-06-01    기타 미용업
2016-06-01    기타 미용업
2016-06-02    기타 미용업
               ...  
2019-02-28    기타 미용업
2019-02-28    기타 미용업
2019-02-28    기타 미용업
2019-02-28    기타 미용업
2019-02-28    기타 미용업
Name: type_of_business, Length: 4481, dtype: object

In [132]:
pd.concat([pd.DataFrame(),df_train[df_train.store_id==0].type_of_business],axis=1)

,type_of_business
2016-06-01,기타 미용업
2016-06-01,기타 미용업
2016-06-01,기타 미용업
2016-06-01,기타 미용업
2016-06-02,기타 미용업
...,...
2019-02-28,기타 미용업
2019-02-28,기타 미용업
2019-02-28,기타 미용업
2019-02-28,기타 미용업


In [144]:
def resample_day(train_df):
    df_day = pd.DataFrame()
    for i in train_df.store_id.unique():
        df_num = train_df[train_df.store_id == i]  # store_id별 데이터

        count_cols = df_num['card_id'].resample(rule='d').count().rename('num_of_pay')  # 일별로 card_id의 개수를 카운팅해서 '일 거래 횟수' 확인
        # 'card_id' value count가 2보다 크면 단골인 것으로 판단하고 단골 방문 횟수 확인
        revisit_idx = df_num.card_id.value_counts().reset_index().query("card_id > 2")["index"].values   # value count가 2보다 큰 card_id를 가져옴.
        revisit_ct = df_num[df_num.card_id.isin(revisit_idx)].card_id.resample(rule='d').count().rename('num_of_revisit')    # 단골 방문횟수 
        sum_cols = df_num[['installment_term', 'amount']].resample(rule='d').sum() # 할부 개월수와 매출액은 일 단위로 합

        df_num_day = pd.concat([count_cols, revisit_ct, sum_cols], axis=1)

        df_num_day.insert(0, 'store_id', i)   # 첫번째 컬럼에 store_id를 삽입
        df_num_day.insert(4, 'region', df_num[df_num.store_id == i].region.unique()[0])    # 다섯번째 컬럼에 region을 삽입
        df_num_day.insert(5, 'type_of_business', df_num[df_num.store_id == i].type_of_business.unique()[0])  # 여섯번째 컬럼에 type_of_business 삽입

        df_day = pd.concat([df_day, df_num_day], axis=0)   # 데이터프레임 concat
        
    df_day.insert(1, 'day_of_week', df_day.index.dayofweek)
    df_day.insert(2, 'business_day', df_day.day_of_week.replace({0:1, 2:1, 3:1, 4:1, 5:0, 6:0}).values)
    df_day.num_of_revisit.fillna(0, inplace=True)
    
    return df_day

In [145]:
%%time
df_day = resample_day(df_train)

CPU times: user 2min 35s, sys: 1min 24s, total: 3min 59s
Wall time: 4min 5s


In [146]:
df_day.head()

NameError: name 'df_day' is not defined

In [ ]:
resample_day(df_train)

In [ ]:
df_day.isnull().sum()

In [ ]:
df_day.to_csv('./data/funda_train_day.csv')

In [ ]:
df_day = pd.read_csv('./data/funda_train_day.csv')
df_day['transacted_date'] = pd.to_datetime(df_day['transacted_date'])
df_day = df_day.set_index('transacted_date')

각 column간 상관 관계 확인

### Modeling을 위해 Month 단위로 resampling

In [ ]:
df_day.head(3)

In [ ]:
def resample_month(frame_day):
    sum_cols = ['num_of_pay', 'num_of_revisit', 'installment_term', 'amount']

    df_monthly = pd.DataFrame()

    for i in frame_day.store_id.unique():
        df_set = frame_day[frame_day.store_id == i]
        
        # nan값이 발생하는 경우를 없애기 위해 이전, 이후 달에 대한 정보를 추가한 후 제거
        prev_date = pd.date_range(start=(df_set.index[0] - relativedelta(months=1)), end=(df_set.index[0] - relativedelta(months=1)))
        add_date = pd.date_range(start=(df_set.index[-1] + relativedelta(months=1)), end=(df_set.index[-1] + relativedelta(months=1)))
        df_set = pd.concat([pd.DataFrame(index=prev_date), df_set, pd.DataFrame(index=add_date)], axis=0)

        df_set.loc[dt.strftime(df_set.index[0], '%Y-%m'), :] = 1
        df_set.loc[dt.strftime(df_set.index[-1], '%Y-%m'), :] = 1

        tot_day = df_set[df_set.amount != 0].day_of_week.resample(rule='m').count().rename('real_tot_day')
        business = df_set[df_set.amount != 0].business_day.resample(rule='m').sum().rename('real_business_day')

        business = business.drop([business.index[0], business.index[-1]], axis=0)
        tot_day = tot_day.drop([tot_day.index[0], tot_day.index[-1]], axis=0)
        df_set = df_set.drop([df_set.index[0], df_set.index[-1]], axis=0)

        df = pd.concat([tot_day, business, df_set[sum_cols].resample(rule='m').sum()], axis=1)

        df.insert(0, 'store_id', i)
        df.insert(6, 'region', df_set.region.values[0])
        df.insert(7, 'type_of_business', df_set.type_of_business.values[0])

        df_monthly = pd.concat([df_monthly, df], axis=0)
   
    return df_monthly